# - Playing LDA on the arxiv -
If you have sklls on this and/or are interested in contributing, you are welcome!

feedback: sequi@unsam.edu.ar Ezequiel Alvarez

## Set up

In [ ]:
import pandas as pd
import pickle
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np
np.random.seed(2018)
from random import randint
from operator import add
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## Load Saved Model

In [ ]:
lda_model = gensim.models.LdaMulticore.load("models/hep-ph")

In [ ]:
data = pd.read_pickle('data/data_hep-ph.pickle');
with open ('data/bow_corpus_hep-ph.pickle', 'rb') as file:
    bow_corpus = pickle.load(file)

In [ ]:
Ntopics=0;
for idx, topic in lda_model.print_topics(-1):
    Ntopics=Ntopics+1;
print(Ntopics)

## Learn your preferences 

In [ ]:
topicscore = [0] * Ntopics
for i in range(20):
    docnumber = randint(0,len(bow_corpus))
    print("\n"+color.BOLD+data.iloc[docnumber,2]+color.END+"\n\n"+data.iloc[docnumber,3])
    score = int(input("Score this paper between -10 (worst) and +10 (best): "))
    score2add = [row[1]*score for row in lda_model[bow_corpus[docnumber]]]
    if ( len(score2add) != Ntopics ):
        print('Not good length')
    if ( len(score2add) == Ntopics ):
        topicscore=list( map(add, topicscore, score2add) )
        print("----")
     
#print("\n\nGood, your score over topics is:\n")
#print(topicscore)
topicscore = np.array(topicscore)

### Game 1: Sort 10 random papers according to your preferences

In [ ]:
out = pd.DataFrame(columns=['score', 'title'])
for i in range(10):
    docnumber = randint(0,len(bow_corpus))
    thistitle=np.array([row[1] for row in lda_model[bow_corpus[docnumber]]])
    if ( len(thistitle) == Ntopics ):
        thisscore=thistitle.dot(topicscore)
    out = out.append({'score': thisscore, 'title':data.iloc[docnumber,2]},ignore_index=True)
pd.options.display.max_colwidth = 150
display(out[['title']])
input("Press [enter] to see how I guess your preferences!")
display(out.sort_values('score',ascending=False))


# Game 2: Match between 2 papers

In [ ]:
    for hh in range(10):
     docnumber1 = randint(0,len(bow_corpus))
     docnumber2 = randint(0,len(bow_corpus))     
     thistitle1=np.array([row[1] for row in lda_model[bow_corpus[docnumber1]]])
     thistitle2=np.array([row[1] for row in lda_model[bow_corpus[docnumber2]]])
     print("Paper A: "+color.BOLD+data.iloc[docnumber1,2]+color.END+'\n'+data.iloc[docnumber1,3]+'\n')
     print("Paper B: "+color.BOLD+data.iloc[docnumber2,2]+color.END+'\n'+data.iloc[docnumber2,3]+'\n')
     input("Press [enter] to see how I guess your preferences!")
     print ("\nThis is my preference:\n")
     score1=thistitle1.dot(topicscore)
     score2=thistitle2.dot(topicscore)
     if (score1 > score2):
            print("1)"+data.iloc[docnumber1,2]+"\t (Score: "+str(round(score1,2))+")")
            print("2)"+data.iloc[docnumber2,2]+"\t (Score: "+str(round(score2,2))+")"+"\n--------------\n")
     else:
            print("1)"+data.iloc[docnumber2,2]+"\t (Score: "+str(round(score2,2))+")")
            print("2)"+data.iloc[docnumber1,2]+"\t (Score: "+str(round(score1,2))+")"+"\n--------------\n")
     input("Press [enter] to see next papers")

# Bonus Track: get your most interesting papers in the Corpus!

In [ ]:
allscores=[]
for docnumber in range(len(data)):
    thistitle=np.array([row[1] for row in lda_model[bow_corpus[docnumber]]])
    thisscore=thistitle.dot(topicscore)
    allscores.append(thisscore)
data['score'] = allscores
sorted_data=data.sort_values('score',ascending=False)

In [ ]:
display(sorted_data[['title','authors','score']][:20])